In [1]:
import torch
# import triton
# import triton.language as tl
import exact.cpp_extension.quantization as ext_quantization
import time
import sys
# sys.path.append('..')
# from triton_layer.layers import TritonDropout, TritonReLU
# import triton_layer.pack_mask_layers as pack_mask_layers
import exact.layers as exact_layers
import tabulate

/home/akai/anaconda3/envs/EXACT2/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
class QDropoutNet(torch.nn.Module):
    def __init__(self, in_channels, out_channels, dropout):
        super().__init__()
        self.in_channels = in_channels
        self.out_channels = out_channels
        self.dropout = dropout
        self.dropout_layer = exact_layers.QDropout(dropout)
    def forward(self, input):
        return self.dropout_layer(input)
    
class QLowMemDropoutNet(torch.nn.Module):
    def __init__(self, in_channels, out_channels, dropout):
        super().__init__()
        self.in_channels = in_channels
        self.out_channels = out_channels
        self.dropout = dropout
        self.dropout_layer = exact_layers.QLowMemDropout(dropout)
    def forward(self, input):
        return self.dropout_layer(input)

In [3]:


def test_speed(nets, input, runtime=1000):
    net_num = len(nets)
    net_forward_time = [0.0] * net_num
    net_backward_time = [0.0] * net_num
    for _ in range(runtime):
        for i, (netname, net) in enumerate(nets):
            start = time.time()
            output = net(input)
            end = time.time()
            net_forward_time[i] += end - start
            loss = output.sum()
            start = time.time()
            loss.backward()
            end = time.time()
            net_backward_time[i] += end - start
    netnames = [netname for netname, _ in nets]
    # for i, (netname, _) in enumerate(nets):
    net_forward_time = [i/runtime*1e6 for i in net_forward_time]
    net_backward_time = [i/runtime*1e6 for i in net_backward_time]
    print(tabulate.tabulate([
        ['']+netnames,
        ["forward avg (us)"] + net_forward_time,
        ["backward avg (us)"] + net_backward_time,
    ]))
    # print(tabulate.tabulate([
    #     ['']+netnames[5:],
    #     ["forward avg (us)"] + net_forward_time[5:],
    #     ["backward avg (us)"] + net_backward_time[5:],
    # ]))
        # print(f'{netname} forward avg time: {net_forward_time[i] / runtime*1e6} us')
        # print(f'{netname} backward avg time: {net_backward_time[i] / runtime*1e6} us')
    

In [7]:
N = 512
input = torch.rand(N, N, dtype=torch.float32, requires_grad=True).cuda()
nets = [
    # ('triton dropout', TDropoutNet(N, N, 0.5).cuda()),
    # ('triton pack mask dropout', TPackMaskDropoutNet(N, N, 0.5).cuda()),
    # ('triton low mem dropout', TLowMemDropoutNet(N, N, 0.5).cuda()),
    # ('torch dropout', TorchDropoutNet(N, N, 0.5).cuda()),
    ('exact pack mask dropout', QDropoutNet(N, N, 0.5).cuda()),
    ('exact low mem dropout',  QLowMemDropoutNet(N, N, 0.5).cuda()),
    # ('triton relu', TReluNet(N, N).cuda()),
    # ('triton pack mask relu', TPackMaskReluNet(N, N).cuda()),
    # ('torch relu', TorchReluNet(N, N).cuda()),
    # ('exact pack mask relu', QReluNet(N, N).cuda()),
]
test_speed(nets, input, runtime=10)

RuntimeError: CUDA error: an illegal memory access was encountered
CUDA kernel errors might be asynchronously reported at some other API call,so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1.